<a href="https://colab.research.google.com/github/AnshuVairagade/Mental-Health-ChatBot-/blob/main/Mental_health_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_groq langchain_core langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.0 MB/s eta 0:00:00


In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model_name="llama-3.3-70b-versatile"
)

result = llm.invoke("Who is lord ram")
print(result.content)

Lord Rama, also known as Ramachandra, is a major deity in Hinduism and the seventh avatar (incarnation) of the god Vishnu. He is considered one of the most important and revered figures in Hindu mythology and is often referred to as the "Maryada Purushottama" or the "Perfect Man".

According to the Hindu epic, the Ramayana, Lord Rama was born in the city of Ayodhya, in the kingdom of Kosala, to King Dasharatha and Queen Kausalya. He was the eldest of four brothers, including Lakshmana, Bharata, and Shatrughna.

The story of Lord Rama is as follows:

* Lord Rama was exiled to the forest for 14 years by his stepmother, Queen Kaikeyi, who wanted her own son, Bharata, to become the king.
* During his exile, Lord Rama's wife, Sita, was abducted by the demon king Ravana, who took her to his kingdom in Lanka (modern-day Sri Lanka).
* Lord Rama, along with his brother Lakshmana and the monkey god Hanuman, embarked on a journey to rescue Sita and defeat Ravana.
* After a long and arduous battle

In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 7.0 MB/s eta 0:00:00


In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.8 MB/s eta 0:0

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
import os

def initialize_llm():
    llm = ChatGroq(
        temperature=0,
        groq_api_key="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",
        model_name="llama-3.3-70b-versatile"
    )
    return llm

def create_vector_db():
    loader = DirectoryLoader('/content/data/', glob='*.pdf', loader_cls=PyPDFLoader)  # Fixed path
    documents = loader.load()
    textSplitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = textSplitter.split_documents(documents)
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
    vector_db.persist()
    print("Chroma DB created")
    return vector_db

def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt_template = """ You are a compassionate mental health chatbot,
    Respond thoroughly to the following question
    {context}
    Question: {question}
    Chatbot: """
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain

def main():
    print("Initializing LLM...")
    llm = initialize_llm()

    print("Creating vector database...")
    db_path = "./chroma_db"
    if not os.path.exists(db_path):
        vector_db = create_vector_db()
    else:
        embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
        vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

    print("Setting up QA chain...")
    qa_chain = setup_qa_chain(vector_db, llm)

    while True:
        query = input("\nHuman: ")
        if query.lower() == "exit":
            print("Chatbot: Take Care of Yourself, GoodBye!")
            break
        response = qa_chain.invoke({"query": query})
        print("\nChatbot:")
        print(response['result'])

if __name__ == "__main__":
    main()

Initializing LLM...
Creating vector database...


<ipython-input-2-c15ff42e1c8e>:54: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-2-c15ff42e1c8e>:55: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)


Setting up QA chain...

Human: Hello

Chatbot:
Hello! It's wonderful to connect with you. I'm here to listen, support, and help in any way I can. Please know that this is a safe and non-judgmental space for you to express yourself. If you're feeling overwhelmed, struggling with difficult emotions, or just need someone to talk to, I'm here for you.

Before we begin, I want to assure you that our conversation will remain confidential and anonymous. You can share as much or as little as you feel comfortable with, and we can take things at your own pace.

If you're not sure where to start or what's been on your mind lately, that's perfectly okay. We can begin with some gentle and open-ended questions to help us get started. Alternatively, if there's something specific that's been weighing on your mind, feel free to share it with me, and I'll do my best to provide a supportive and empathetic ear.

Some things you might want to consider sharing with me could include:

* How you've been feeli

KeyboardInterrupt: Interrupted by user

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 80.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
import os
import gradio as gr

def initialize_llm():
    llm = ChatGroq(
        temperature=0,
        groq_api_key="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",
        model_name="llama-3.3-70b-versatile"
    )
    return llm

def create_vector_db():
    loader = DirectoryLoader('/content/data/', glob='*.pdf', loader_cls=PyPDFLoader)  # Fixed path
    documents = loader.load()
    textSplitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = textSplitter.split_documents(documents)
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
    vector_db.persist()
    print("Chroma DB created")
    return vector_db

def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt_template = """You are a compassionate mental health chatbot. Respond thoroughly to mental health-related questions. If the question is not related to mental health, say "I don't know."
    {context}
    Question: {question}
    Chatbot: """
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain


print("Initializing LLM...")
llm = initialize_llm()

print("Creating vector database...")
db_path = "./chroma_db"
if not os.path.exists(db_path):
    vector_db = create_vector_db()
else:
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

print("Setting up QA chain...")
qa_chain = setup_qa_chain(vector_db, llm)

def chatbot_response(user_input, history=[]):
    if not user_input.strip():
        return "Provide valid input", history
    response = qa_chain.invoke(user_input)
    history.append((user_input, response["result"]))
    return "", history

# Gradio Frontend
with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("# Mental Health Chatbot")
    chatbot = gr.Chatbot(label="Chat")
    msg = gr.Textbox(label="Your Message", placeholder="Type your message here...")
    clear = gr.Button("Clear")

    def respond(message, chat_history):
        if not message.strip():
            return "Provide valid input", chat_history
        response = qa_chain.invoke(message)
        chat_history.append((message, response["result"]))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

app.launch(share=True, debug=True)

Initializing LLM...
Creating vector database...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

<ipython-input-4-cce319a44e50>:56: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)


Setting up QA chain...


/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:284: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://06e05db8ebd2d11f49.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://06e05db8ebd2d11f49.gradio.live


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
import os

# Initialize LLM
def initialize_llm():
    llm = ChatGroq(
        temperature=0,
        groq_api_key="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",
        model_name="llama-3.3-70b-versatile"
    )
    return llm

# Create vector database
def create_vector_db():
    loader = DirectoryLoader('/content/data/', glob='*.pdf', loader_cls=PyPDFLoader)
    documents = loader.load()

    # Ensure metadata includes the filename
    for doc in documents:
        doc.metadata["source"] = doc.metadata.get("source", "Unknown File")

    # Split documents into smaller chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)

    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
    vector_db.persist()

    print("✅ Chroma DB Created Successfully")
    return vector_db

# Set up QA chain with improved response formatting
def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()

    # Improved Prompt for Response Formatting
    prompt_template = """You are a mental health chatbot providing well-structured responses.
    Respond thoroughly using the context provided. If the question is not related to mental health, say "I don't know."

📝 **Answer:**
{context}
**Question:** {question}
**Chatbot:** """

    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": PROMPT}
    )

    return qa_chain

# Main function
def main():
    print("🚀 Initializing LLM...")
    llm = initialize_llm()

    print("📂 Creating Vector Database...")
    db_path = "./chroma_db"
    if not os.path.exists(db_path):
        vector_db = create_vector_db()
    else:
        embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
        vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

    print("🤖 Setting Up QA Chain...")
    qa_chain = setup_qa_chain(vector_db, llm)

    # Interactive Chat Loop
    while True:
        query = input("\n🗣️ Human: ")
        if query.lower() == "exit":
            print("💙 Chatbot: Take care of yourself. Goodbye!")
            break

        # Get response
        response = qa_chain.invoke({"query": query})

        # Formatting Output with References
        print("\n🤖 **Chatbot:**\n")
        print(response['result'])



if __name__ == "__main__":
    main()


🚀 Initializing LLM...
📂 Creating Vector Database...
🤖 Setting Up QA Chain...

🗣️ Human: I am feeling depressed 

🤖 **Chatbot:**

I'm so sorry to hear that you're feeling depressed. It takes a lot of courage to acknowledge and share your emotions, and I'm here to listen and support you.

Depression can be a challenging and overwhelming experience, but please know that you're not alone. It's estimated that millions of people worldwide suffer from depression, and there is help available.

Can you tell me more about what you're experiencing? How long have you been feeling this way, and what are some of the symptoms you're struggling with? Are you having trouble sleeping, experiencing changes in appetite, or feeling hopeless or disconnected from activities you once enjoyed?

It might also be helpful to explore some coping strategies and techniques that can help you manage your depression. Some people find it helpful to engage in regular exercise, practice mindfulness or meditation, or conne